In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

In [16]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
#from sklearn.xgboost import XGBRegressor
from sklearn import linear_model

In [ ]:
pd.read_csv('/Users/mwilson/Documents/GitHub/insurance/data/')